# AFAP Orchestrator Validation Notebook
## Profile-by-Profile Simulation & Contract Assurance

### Purpose
This notebook validates the AFAP orchestrator across all supported
Analysis Interpretation Frameworks (AIFs).

Each profile is treated as an **independent client case**, ensuring:
- Correct engine selection
- Stable output contracts
- Profile-aware AI interpretation
- Reproducible, auditable execution

This notebook is intentionally structured to support:
- Engineering validation
- Audit defensibility
- Investor / stakeholder review


In [1]:
# ---------------------------------------------------------------
# Environment setup
# ---------------------------------------------------------------

import sys
import os
import pandas as pd
from pprint import pprint

# Ensure project root is on PYTHONPATH
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root resolved to:", PROJECT_ROOT)


Project root resolved to: c:\Users\ADMIN\Documents\My Documents\MyDataAnalysis\Financial statement analysis\financial-analysis-pipeline


### Step 1: Import AFAP Orchestrator & Frozen Contracts

This cell imports:
- The AFAP orchestrator
- The frozen output contract
- All supported analysis profiles

These are treated as **non-negotiable system interfaces**.


In [2]:
from orchestrator.orchestrator import (
    afap_run,
    ANALYSIS_PROFILES,
    AFAP_OUTPUT_KEYS
)

pprint(ANALYSIS_PROFILES)


{'full_diagnostic': {'engines': ['ratio',
                                 'trend',
                                 'cash_flow',
                                 'anomaly',
                                 'solvency',
                                 'composite_risk'],
                     'metrics_scope': 'all'},
 'going_concern_screen': {'engines': ['ratio',
                                      'trend',
                                      'solvency',
                                      'composite_risk'],
                          'metrics_scope': 'critical_only'},
 'liquidity_focus': {'engines': ['ratio',
                                 'trend',
                                 'cash_flow',
                                 'composite_risk'],
                     'metrics_scope': ['current_ratio',
                                       'quick_ratio',
                                       'cash_ratio']},
 'performance_focus': {'engines': ['ratio', 'trend'],
                    

### Step 2: Load Financial Statements

This dataset simulates a **single recurring client**
across multiple years, allowing trend and risk analysis.


In [ ]:
financials_df = pd.read_csv("../data/cleaned/financial_statements.csv")
financials_df.head()


,Company,Year,FS Category,FS Subcategory,Statement,Amount
0,Acme Manufacturing Ltd,2020,Assets,Current Assets,Balance Sheet,3109667
1,Acme Manufacturing Ltd,2020,Assets,Non-Current Assets,Balance Sheet,905812
2,Acme Manufacturing Ltd,2020,Equity,Equity,Balance Sheet,2152630
3,Acme Manufacturing Ltd,2020,Expenses,COGS,Income Statement,373114
4,Acme Manufacturing Ltd,2020,Expenses,Finance Costs,Income Statement,2929304


### Step 3: Define Validation Functions

These validators ensure:
- Output contract integrity
- AI interpretation structural consistency

They deliberately avoid business logic
and only validate **system correctness**.


In [4]:
def validate_output_contract(outputs: dict):
    """
    Ensures all required AFAP output keys are present.
    """
    missing = set(AFAP_OUTPUT_KEYS) - set(outputs.keys())
    assert not missing, f"Missing output keys: {missing}"


def validate_ai_interpretation(ai_output):
    """
    Validates AI interpretation structure (schema-level).
    """
    assert isinstance(ai_output, list)
    assert len(ai_output) > 0

    for rec in ai_output:
        assert "Company" in rec
        assert "Year" in rec
        assert "interpretation" in rec


### Step 4: Define Independent Client Analysis Cases

Each profile below represents a **realistic engagement scenario**:
- Targeted diagnostics
- Focused decision support
- Reduced computational scope

Each will be run **independently**.


In [5]:
CLIENT_CASES = [
    "full_diagnostic",
    "solvency_focus",
    "liquidity_focus",
    "performance_focus",
    "risk_scan",
    "going_concern_screen"
]


## Step 5: Execute a Single AFAP Profile (Case Simulation)

This cell is the **canonical execution pattern**.
It will be reused for every profile.

We:
1. Run the orchestrator
2. Validate the output contract
3. Validate AI interpretation
4. Inspect executed engines
5. Review sample outputs


In [6]:
# Select profile to simulate
profile_name = "solvency_focus"

print(f"\n--- Running AFAP Case: {profile_name} ---")

outputs = afap_run(
    financials_df=financials_df,
    analysis_profile=profile_name,
    use_mock_ai=False  # toggle True for fast dry-runs
)

# --- Contract validation ---
validate_output_contract(outputs)
print("✅ Output contract validated")

# --- AI validation ---
validate_ai_interpretation(outputs["ai_interpretation"])
print("✅ AI interpretation structure validated")

# --- Inspect executed engines ---
engines_ran = [
    k for k in AFAP_OUTPUT_KEYS
    if k in outputs and outputs[k]
]

print("Engines executed:", engines_ran)



--- Running AFAP Case: solvency_focus ---
✅ ratio_engine output validated successfully.
✅ solvency_engine output validated successfully.
✅ Output contract validated
✅ AI interpretation structure validated
Engines executed: ['profile_used', 'ratios', 'solvency', 'ai_interpretation']


In [11]:
outputs

{'ratios': [{'engine': 'ratio_engine',
   'Company': 'Acme Manufacturing Ltd',
   'Year': 2020,
   'metrics': {'current_ratio': np.float64(1.5876421245028514),
    'quick_ratio': np.float64(1.5876421245028514),
    'gross_margin': np.float64(0.8673714331416791),
    'operating_margin': np.float64(0.28910663029085837),
    'net_margin': np.float64(-1.5461482817762533),
    'debt_equity': np.float64(0.9825237035626188),
    'interest_coverage': np.float64(0.2776502541217982),
    'asset_turnover': np.float64(0.7005951220265378),
    'roa': np.float64(-1.0832239441421558),
    'roe': np.float64(-2.020627325643515)},
   'flags': {},
   'severity': 'stable',
   'explanation': 'Canonical financial ratios'},
  {'engine': 'ratio_engine',
   'Company': 'Acme Manufacturing Ltd',
   'Year': 2021,
   'metrics': {'current_ratio': np.float64(3.0794864428132476),
    'quick_ratio': np.float64(3.0794864428132476),
    'gross_margin': np.float64(0.8548843465641977),
    'operating_margin': np.float64(0

### Step 6: Inspect AI Interpretation Output

This validates **semantic usability**, not correctness.


In [9]:
sample_interp = outputs["ai_interpretation"][0]

print("Company:", sample_interp["Company"])
print("Year:", sample_interp["Year"])
print("\n--- AI Interpretation ---\n")
print(sample_interp["interpretation"])


Company: Acme Manufacturing Ltd
Year: 2020

--- AI Interpretation ---

summary:
Acme Manufacturing Ltd — 2020 solvency-focused summary. Current and quick ratios are 1.588 (equal values) indicating short-term liquidity marginally above a conservative 1.50 threshold. Gross margin (86.737%) and operating margin (28.911%) are materially positive and exceed conservative profitability thresholds. However, net margin is negative (−154.615%), return on assets (−108.322%) and return on equity (−202.063%) are substantially adverse. Debt-to-equity is 0.983 (approximately 0.98). Interest coverage is very low at 0.278, indicating operating income is insufficient to meet interest expense. Asset turnover is 0.701. Overall, liquidity at the current/current-asset level appears acceptable by conservative liquidity cut-offs, but solvency and profitability at the net and return levels are concerning given the negative net income and inadequate interest coverage.

key_risks:
- Insufficient interest coverag

### Step 7: Inspect Numerical Engine Outputs

This confirms:
- Ratios exist
- Engines populated data
- Schema consistency


In [13]:
if outputs.get("ratios"):
    display(pd.DataFrame(outputs["ratios"]).head())

if outputs.get("solvency"):
    display(pd.DataFrame(outputs["solvency"]).head())


,engine,Company,Year,metrics,flags,severity,explanation
0,ratio_engine,Acme Manufacturing Ltd,2020,"{'current_ratio': 1.5876421245028514, 'quick_r...",{},stable,Canonical financial ratios
1,ratio_engine,Acme Manufacturing Ltd,2021,"{'current_ratio': 3.0794864428132476, 'quick_r...",{},stable,Canonical financial ratios
2,ratio_engine,Acme Manufacturing Ltd,2022,"{'current_ratio': 3.4109333800409956, 'quick_r...",{},stable,Canonical financial ratios
3,ratio_engine,Acme Manufacturing Ltd,2023,"{'current_ratio': 4.67288599769455, 'quick_rat...",{},stable,Canonical financial ratios
4,ratio_engine,Banyan Retail Co,2020,"{'current_ratio': 2.513537113097586, 'quick_ra...",{},stable,Canonical financial ratios


,engine,Company,Year,metrics,flags,severity,explanation
0,solvency_engine,Acme Manufacturing Ltd,2020,"{'debt_equity': 0.9825237035626188, 'interest_...","{'high_leverage': False, 'weak_coverage': True}",watch,Capital structure shows solvency risk.
1,solvency_engine,Acme Manufacturing Ltd,2021,"{'debt_equity': 1.3584280927054282, 'interest_...","{'high_leverage': False, 'weak_coverage': True}",watch,Capital structure shows solvency risk.
2,solvency_engine,Acme Manufacturing Ltd,2022,"{'debt_equity': 0.6969581313037093, 'interest_...","{'high_leverage': False, 'weak_coverage': False}",stable,Solvency position acceptable.
3,solvency_engine,Acme Manufacturing Ltd,2023,"{'debt_equity': 1.4765422770808412, 'interest_...","{'high_leverage': False, 'weak_coverage': False}",stable,Solvency position acceptable.
4,solvency_engine,Banyan Retail Co,2020,"{'debt_equity': 1.0475669775251273, 'interest_...","{'high_leverage': False, 'weak_coverage': False}",stable,Solvency position acceptable.


## Step 8: Full System Validation (All Profiles)

This cell simulates **multiple client engagements**,
ensuring no profile breaks the orchestrator.


In [15]:
results_by_profile = {}

for profile_name in CLIENT_CASES:
    print(f"\n--- Executing profile: {profile_name} ---")

    outputs = afap_run(
        financials_df=financials_df,
        analysis_profile=profile_name,
        use_mock_ai=False
    )

    validate_output_contract(outputs)
    validate_ai_interpretation(outputs["ai_interpretation"])

    engines_ran = [
        k for k in AFAP_OUTPUT_KEYS
        if k in outputs and outputs[k]
    ]

    print("Engines executed:", engines_ran)

    results_by_profile[profile_name] = outputs

print("\n✅ All AFAP profiles validated successfully")



--- Executing profile: full_diagnostic ---
✅ ratio_engine output validated successfully.
✅ trend_engine output validated successfully.
✅ cash_flow_engine output validated successfully.
✅ anomaly_efficiency_engine output validated successfully.
✅ solvency_engine output validated successfully.
Engines executed: ['profile_used', 'ratios', 'trend', 'cash_flow', 'anomaly', 'solvency', 'composite_risk', 'ai_interpretation']

--- Executing profile: solvency_focus ---
✅ ratio_engine output validated successfully.
✅ solvency_engine output validated successfully.
Engines executed: ['profile_used', 'ratios', 'solvency', 'ai_interpretation']

--- Executing profile: liquidity_focus ---
✅ ratio_engine output validated successfully.
✅ trend_engine output validated successfully.
✅ cash_flow_engine output validated successfully.
Engines executed: ['profile_used', 'ratios', 'trend', 'cash_flow', 'composite_risk', 'ai_interpretation']

--- Executing profile: performance_focus ---
✅ ratio_engine output v

### Step 9: Compare AI Interpretations Across Profiles

This demonstrates the **profile-aware reasoning differences**.


In [19]:
for profile, outputs in results_by_profile.items():
    interp = outputs["ai_interpretation"][0]
    print(f"\n=== {profile.upper()} ===")
    print(interp["interpretation"][:1200], "...")



=== FULL_DIAGNOSTIC ===
summary:
Acme Manufacturing Ltd — 2020 financial ratio diagnostic. Liquidity appears marginally adequate on a textbook current and quick ratio of 1.59, which is just above a conservative liquidity threshold. Profitability and returns are materially weak: gross margin is high at 86.74% but operating margin is 28.91% while net margin is negative at -154.61%, producing negative return on assets (ROA -1.08%) and return on equity (ROE -2.02%). Leverage is near a conservative limit with a debt-to-equity ratio of 0.98. Interest coverage is very low at 0.28, indicating operating earnings do not cover interest expense. Asset turnover is 0.70. Overall, the ratios indicate adequate short-term liquidity but significant earnings and solvency stress at the net income and interest-coverage level.

key_risks:
- Net profitability breach: net margin = -154.61% (conservative threshold: 0%). The company reported a material net loss relative to revenue in 2020.
- Interest coverage 